In [ ]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-8d760645-f8c8-7110-dda0-e5ce6a72d45b)


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  proj_dir = "/content/drive/MyDrive/ece884_project/"
else:
  proj_dir = "../"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv(f"{proj_dir}data_clean/clean.csv")
column_names = df.columns
df = df.to_numpy()

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(df) # dont forget we have to transfer them back
df = scaler.transform(df)

In [ ]:

generator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(400, activation="selu"),

    keras.layers.BatchNormalization(),
    keras.layers.Dense(400, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(98, activation="sigmoid")
])

In [ ]:

discriminator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(400, activation="selu"),
    keras.layers.BatchNormalization(),

    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
gan = keras.models.Sequential([generator, discriminator])

discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [ ]:
from tqdm import tqdm

In [ ]:
def train_gan(
    gan, dataset, n_epochs=100, iterations=1000, n_noise=20000
):

    """
    Inputs: 

    gan, this is a keras gan object made by combining two neural nets and
    restricting the trainability of one of them.

    dataset, this takes in regular tabular data. now this is training rowwise
    however i may change this to matrix wise like a picture.

    n_epochs, numper of times the gans go though training iterations

    iterations, number of times in gan iteraton loop, 
    it would be a good idea to reduct this after the warmup period

    n_noise, this is the size of fake data generated

    
    Output:

    generators_saved, this is an iterable list of keras objects that can be used
    
    discriminators_saved, same thing, these can be used to test

    for generator, discriminator in zip(gen, desc):
        noise = tf.random.normal(shape=dims)
        generated_data = generator(noise)
        judgement = discriminator(generated_data) # probs data is real
    """
    generator, discriminator = gan.layers
    generators_saved = []
    discriminators_saved = []
    
    for epoch in range(n_epochs):
        print(f"Epoch {epoch} of {n_epochs}")
        if epoch > n_epochs / 4:  # give a 25% training period

            generators_saved.append(generator)
            discriminators_saved.append(discriminator)
            iterations = np.ceil(iterations/ 2).astype('int') + 5 # after the training period reduce iterations

        for _ in tqdm(range(iterations)):
            random_index = np.random.permutation(len(dataset))
            X_batch = dataset[random_index, :]
            # phase 1 - training the discriminator

            mu = tf.random.uniform([1], minval=-1, maxval=1.0)
            sigma = tf.random.uniform([1], minval=0.1, maxval=2.0)
            noise = tf.random.normal(shape=(n_noise, 98),
                                     mean=mu,
                                     stddev=sigma) 

            # maybe increase the sd on this,
            # how about a random sd.love it

            generated_data = generator(noise)
            X_fake_and_real = tf.concat([generated_data, X_batch], axis=0)

            y1 = tf.constant([[0.0]] * noise.shape[0] + [[1.0]] * X_batch.shape[0])
            y1 = np.reshape(y1, (len(y1), 1))

            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            # phase 2 - training the generator
            y2 = tf.constant([[1.0]] * n_noise)
            noise = tf.random.normal(shape=generated_data.shape)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)

    return generators_saved, discriminators_saved

In [ ]:
generators_saved, discriminators_saved = train_gan(
    gan, df, n_epochs=100, iterations=2000, n_noise=5000
)

Epoch 0 of 100


100%|██████████| 2000/2000 [02:42<00:00, 12.30it/s]


Epoch 1 of 100


100%|██████████| 2000/2000 [02:33<00:00, 12.99it/s]


Epoch 2 of 100


...

100%|██████████| 2000/2000 [02:27<00:00, 13.55it/s]


Epoch 47 of 100


100%|██████████| 2000/2000 [02:28<00:00, 13.49it/s]


Epoch 48 of 100


 21%|██        | 422/2000 [00:31<01:55, 13.68it/s]

In [ ]:
import os
import re
import pickle
models = os.listdir(f"{proj_dir}saved_models/list_of_models/gen")
model_number = [int(re.sub("generators", "", x)) for x in models]
last_model = max(model_number)

In [ ]:
with open(f"{proj_dir}saved_models/list_of_models/gen/generators{last_model+1}", "wb") as fp:
    pickle.dump(generators_saved, fp)

with open(f"{proj_dir}saved_models/list_of_models/disc/discriminators{last_model+1}", "wb") as fp:
    pickle.dump(discriminators_saved, fp)